Concepts taught:


Boolean Model :
1.  Inverted Index
2.  Query Processing
3.  TD matrix construction

Plagiarism checker: VSM
1. TF , IDF , TF-IDF
2. Similarity score based Retrival - cosine,jaccard,dice

Probabilistic models:
  1. BIM



NOTE:
Input will be like multiple .txt files with few lines of teaxt in each; We have to consider each as a document and proceed;




In [ ]:
import re
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    stop_words = ['the', 'a', 'is', 'in', 'of', 'and', 'to', 'it', 'for', 'that']
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

# --- Example Usage ---
sample_text = "Here is an example sentence! It shows how preprocessing, like the removal of stopwords and numbers 123, works."
processed_text = basic_preprocess_text(sample_text)

print(f"Original Text:\n'{sample_text}'")
print("-" * 20)
print(f"Processed Text:\n'{processed_text}'")
# Memory Tip: Think "CLEANUP" - like cleaning a messy text!

Original Text:
'Here is an example sentence! It shows how preprocessing, like the removal of stopwords and numbers 123, works.'
--------------------
Processed Text:
'here an example sentence shows how preprocessing like removal stopwords numbers works'


In [ ]:
import os

# Create a directory to store dummy documents if it doesn't exist
if not os.path.exists('documents'):
    os.makedirs('documents')

# Create some dummy text files for demonstration
doc1_content = "This is the first document about information retrieval."
doc2_content = "The second document discusses vector space models."
doc3_content = "Information retrieval models include probabilistic models."

with open('documents/doc1.txt', 'w') as f:
    f.write(doc1_content)

with open('documents/doc2.txt', 'w') as f:
    f.write(doc2_content)

with open('documents/doc3.txt', 'w') as f:
    f.write(doc3_content)

print("Dummy documents created in the 'documents' directory.")

Dummy documents created in the 'documents' directory.


In [ ]:
import os
import pandas as pd

# Define the path to your text files
directory_path = '/content/documents'

# Get a sorted list of the filenames. Sorting ensures a consistent order.
file_list = sorted([f for f in os.listdir(directory_path) if f.endswith('.txt')])

# Read the content of each file into a list.
# A list comprehension provides a concise way to do this.
texts = [open(os.path.join(directory_path, filename), 'r', encoding='utf-8').read() for filename in file_list]

# Create the DataFrame with two columns: 'doc_id' and 'text'
documents_df = pd.DataFrame({'text': texts})
documents_df = documents_df.reset_index().rename(columns={'index': 'docID'})

# Print the DataFrame to see the result
print(documents_df)

   docID                                               text
0      0  This is the first document about information r...
1      1  The second document discusses vector space mod...
2      2  Information retrieval models include probabili...


In [ ]:
# Apply the preprocessing function to the 'text' column
documents_df['processed_text'] = documents_df['text'].apply(preprocess_text)

# Display the DataFrame with the new processed text column
print("\nDataFrame with Processed Text:")
display(documents_df)


DataFrame with Processed Text:


,docID,text,processed_text
0,0,This is the first document about information r...,this first document about information retrieval
1,1,The second document discusses vector space mod...,second document discusses vector space models
2,2,Information retrieval models include probabili...,information retrieval models include probabili...


An inverted index is a dictionary where each key is a unique term and the value is a set of document IDs where that term appears.

In [ ]:
from collections import defaultdict
import re

def build_inverted_index(documents_df):
    """
    Builds an inverted index from a DataFrame containing document text.
    Assumes the DataFrame has 'doc_id' and 'text' columns.
    """
    inverted_index = defaultdict(list)
    # Iterate through the DataFrame rows
    for index, row in documents_df.iterrows():
        doc_id = row['docID']
        doc_text = row['text'] # Use the original text for preprocessing within the function
        terms = row['processed_text'].split()
        for term in set(terms):
            inverted_index[term].append(doc_id)
    return dict(inverted_index)
inverted_index = build_inverted_index(documents_df)


Boolean Model


In [ ]:
def intersect(p1, p2):
    """
    Finds the intersection of two sorted postings lists.
    :param p1: First postings list.
    :param p2: Second postings list.
    :return: A new list containing documents present in both lists.
    """
    answer = []
    i, j = 0, 0
    while i < len(p1) and j < len(p2):
        if p1[i] == p2[j]:
            answer.append(p1[i])
            i += 1
            j += 1
        elif p1[i] < p2[j]:
            i += 1
        else:
            j += 1
    return answer

def union(p1, p2):
    """
    Finds the union of two sorted postings lists.
    :param p1: First postings list.
    :param p2: Second postings list.
    :return: A new list containing documents from either list, with no duplicates.
    """
    answer = sorted(list(set(p1) | set(p2)))
    return answer

def process_query(query, index):

    query_parts = query.lower().split()

    if "and" in query_parts:
        term1 = query_parts[0]
        term2 = query_parts[2]

        # Get the postings lists for the terms
        p1 = index.get(term1, [])
        p2 = index.get(term2, [])

        # Intersect the lists
        return intersect(p1, p2)

    elif "or" in query_parts:
        term1 = query_parts[0]
        term2 = query_parts[2]

        p1 = index.get(term1, [])
        p2 = index.get(term2, [])

        return union(p1, p2)

    elif "not" in query_parts:
        # A NOT query must be an AND NOT query
        # This is a basic implementation of `text AND NOT data`
        term1 = query_parts[0]
        term2 = query_parts[3]

        p1 = index.get(term1, [])
        p2 = index.get(term2, [])

        # Find documents in the first list that are NOT in the second
        return [doc_id for doc_id in p1 if doc_id not in p2]

    else:
        # Simple single-term query
        return index.get(query.lower(), [])

# Example queries
query1 = "Information AND retrieval"
query2 = "vector OR information"
query3 = "vector AND NOT retrieval"

# Process queries and print results
print(f"Query: '{query1}' -> Documents: {process_query(query1, inverted_index)}")
print(f"Query: '{query2}' -> Documents: {process_query(query2, inverted_index)}")
print(f"Query: '{query3}' -> Documents: {process_query(query3, inverted_index)}")

Query: 'Information AND retrieval' -> Documents: [0, 2]
Query: 'vector OR information' -> Documents: [0, 1, 2]
Query: 'vector AND NOT retrieval' -> Documents: []


In [ ]:
# import pandas as pd
# from sklearn.metrics import precision_score, recall_score, f1_score
# from collections import defaultdict

# # Step 1: Assume a DataFrame with documents and relevance labels
# data = {
#     'document': [
#         "information retrieval is a key topic.",
#         "query depends on the model used.",
#         "this is a very different document.",
#         "another document about information retrieval.",
#         "this is not a relevant document."
#     ],
#     'relevance': [1, 1, 0, 1, 0] # Ground truth: 1 for relevant, 0 for irrelevant
# }
# df = pd.DataFrame(data)

# # Step 2: Simulate a retrieval system for the query "information retrieval"
# query_term = "information retrieval"
# df['retrieved'] = df['document'].apply(lambda x: 1 if query_term in x else 0)

# # Step 3: Calculate the evaluation metrics
# y_true = df['relevance']
# y_pred = df['retrieved']

# precision = precision_score(y_true, y_pred)
# recall = recall_score(y_true, y_pred)
# f1 = f1_score(y_true, y_pred)

# print("Simulated DataFrame with Retrieval Results:")
# print(df)
# print("\n--- Evaluation Metrics ---")
# print(f"Precision: {precision:.2f}")
# print(f"Recall: {recall:.2f}")
# print(f"F1 Score: {f1:.2f}")

Simulated DataFrame with Retrieval Results:
                                        document  relevance  retrieved
0          information retrieval is a key topic.          1          1
1               query depends on the model used.          1          0
2             this is a very different document.          0          0
3  another document about information retrieval.          1          1
4               this is not a relevant document.          0          0

--- Evaluation Metrics ---
Precision: 1.00
Recall: 0.67
F1 Score: 0.80


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

def build_td_matrix(documents_df):
    processed_docs = documents_df['processed_text'].tolist()
    vectorizer = TfidfVectorizer(binary=True, use_idf=False)
    td_matrix = vectorizer.fit_transform(processed_docs)
    terms = vectorizer.get_feature_names_out()
    doc_ids = documents_df['docID'].tolist()

    return pd.DataFrame(td_matrix.toarray(), index=doc_ids, columns=terms)

# Use the documents_df created from reading the text files
# Assuming documents_df has been created and populated and processed_text column exists

td_matrix = build_td_matrix(documents_df)
print("\nTerm-Document Matrix built from DataFrame:")
display(td_matrix) # Use display for better DataFrame output

# The sample documents dictionary and its usage are removed
# Error handling for documents_df and processed_text column is removed as requested.


Term-Document Matrix built from DataFrame:


,about,discusses,document,first,include,information,models,probabilistic,retrieval,second,space,this,vector
0,0.408248,0.000000,0.408248,0.408248,0.000000,0.408248,0.000000,0.000000,0.408248,0.000000,0.000000,0.408248,0.000000
1,0.000000,0.408248,0.408248,0.000000,0.000000,0.000000,0.408248,0.000000,0.000000,0.408248,0.408248,0.000000,0.408248
2,0.000000,0.000000,0.000000,0.000000,0.447214,0.447214,0.447214,0.447214,0.447214,0.000000,0.000000,0.000000,0.000000


VSM ( based on Plagiarism ws)


In [ ]:
import numpy as np

def cosine_similarity_custom(vec1, vec2):
    """
    Calculates the cosine similarity between two vectors.
    """
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)

    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0
    return dot_product / (norm_vec1 * norm_vec2)

def jaccard_similarity(set1, set2):
    """
    Calculates the Jaccard similarity between two sets.
    """
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))

    if union == 0:
        return 0
    return intersection / union

def dice_similarity(set1, set2):
    """
    Calculates the Dice similarity between two sets.
    """
    intersection = len(set1.intersection(set2))
    sum_of_sizes = len(set1) + len(set2)

    if sum_of_sizes == 0:
        return 0
    return (2 * intersection) / sum_of_sizes

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
# Assuming preprocess_text is available from a previous cell

# Assuming td_matrix and documents_df are available from previous cells
# Assuming vectorizer (used to create td_matrix) is also available.
# If vectorizer was created locally in build_td_matrix, you might need to
# modify build_td_matrix to return the vectorizer instance.
# For this code to run as is, 'vectorizer' must be in the global scope or passed in.

# Since the original vectorizer might not be globally available from build_td_matrix,
# I'll recreate and refit a temporary one here for the query transformation.
# In a complete script, it's better to reuse the original vectorizer instance.
temp_vectorizer = TfidfVectorizer(binary=True, use_idf=False)
# Fit on the processed text of the documents to ensure the vocabulary matches the td_matrix
temp_vectorizer.fit(documents_df['processed_text'].tolist())


def calculate_similarity_vsm_query(query_text, td_matrix, vectorizer, documents_df):
    """
    Calculates cosine similarity between a query and documents using the VSM.

    Args:
        query_text (str): The text of the query.
        td_matrix (sparse matrix or array): The Term-Document matrix.
        vectorizer (TfidfVectorizer): The fitted vectorizer used to create the td_matrix.
        documents_df (DataFrame): DataFrame containing original document text and doc_ids.

    Returns:
        DataFrame: A DataFrame of documents ranked by similarity score.
    """
    # Preprocess the query text
    processed_query = preprocess_text(query_text)

    # Transform the processed query using the SAME vectorizer fitted on the documents
    query_vector = vectorizer.transform([processed_query])

    # Calculate cosine similarities between the query vector and the document vectors
    cosine_scores = cosine_similarity(query_vector, td_matrix).flatten()

    # Create a DataFrame to display results
    results_df = pd.DataFrame({
        'doc_id': documents_df['docID'], # Use doc_ids from the original DataFrame
        'text': documents_df['text'],     # Include original text for context
        'similarity_score': cosine_scores
    })

    # Sort the results by similarity score in descending order
    results_df = results_df.sort_values(by='similarity_score', ascending=False)

    return results_df

# --- Example Usage with user input ---
user_query = input("Enter your search query for VSM: ")

# Pass the necessary objects to the function
# Using the temporary vectorizer created above
vsm_results = calculate_similarity_vsm_query(user_query, td_matrix, temp_vectorizer, documents_df)

print(f"\nVSM Results for query: '{user_query}'")
display(vsm_results)

# The original calculate_similarity_vsm function is replaced.

Enter your search query for VSM: vector

VSM Results for query: 'vector'


,doc_id,text,similarity_score
1,1,The second document discusses vector space mod...,0.408248
0,0,This is the first document about information r...,0.000000
2,2,Information retrieval models include probabili...,0.000000


In [ ]:
# --- Probabilistic Model: Binary Independence Model (BIM) ---
import math

def build_inverted_index_bim(documents_df):
    """
    Builds an inverted index from the 'processed_text' column of a DataFrame.
    """
    inverted_index = {}
    for _, row in documents_df.iterrows():
        doc_id = row['docID']
        processed_terms = set(row['processed_text'].split())
        for term in processed_terms:
            if term not in inverted_index:
                inverted_index[term] = []
            inverted_index[term].append(doc_id)
    return inverted_index

def rank_bim_phase1(query_text, documents_df):
    """
    Ranks documents using the Binary Independence Model (BIM) based on
    the initial probabilities without relevance feedback.

    Args:
        query_text (str): The user's search query.
        documents_df (pd.DataFrame): DataFrame with 'docID' and 'processed_text'.

    Returns:
        pd.DataFrame: A DataFrame of documents ranked by their BIM score.
    """
    N = len(documents_df)
    processed_query = set(preprocess_text(query_text).split())

    inverted_index = build_inverted_index_bim(documents_df)

    term_weights = {}
    for term in processed_query:
        if term in inverted_index:
            nk = len(inverted_index[term])
            # Simplified BIM weighting formula for initial ranking
            weight = math.log((N - nk + 0.5) / (nk + 0.5))
            term_weights[term] = weight
        else:
            term_weights[term] = 0

    scores = {}
    for _, row in documents_df.iterrows():
        doc_id = row['docID']
        doc_terms = set(row['processed_text'].split())
        score = sum(term_weights.get(term, 0) for term in processed_query if term in doc_terms)
        scores[doc_id] = score

    results_df = pd.DataFrame(list(scores.items()), columns=['docID', 'score'])
    results_df = results_df.sort_values(by='score', ascending=False).reset_index(drop=True)

    return pd.merge(results_df, documents_df, on='docID')

def rank_bim_phase2(query_text, documents_df, relevant_doc_ids):
    """
    Ranks documents using the Binary Independence Model (BIM) with
    relevance feedback.

    Args:
        query_text (str): The user's search query.
        documents_df (pd.DataFrame): DataFrame with 'docID' and 'processed_text'.
        relevant_doc_ids (list): A list of docIDs that the user has marked as relevant.

    Returns:
        pd.DataFrame: A DataFrame of documents with a refined ranking.
    """
    N = len(documents_df)
    processed_query = set(preprocess_text(query_text).split())

    inverted_index = build_inverted_index_bim(documents_df)

    # Calculate key parameters from relevance feedback
    r = len(relevant_doc_ids)

    term_weights = {}
    for term in processed_query:
        if term in inverted_index:
            nk = len(inverted_index[term])

            # Find the number of relevant documents containing the term
            rk = sum(1 for doc_id in inverted_index[term] if doc_id in relevant_doc_ids)

            # BIM weighting formula with relevance feedback (smoothing applied)
            weight = math.log(
                ((rk + 0.5) / (r - rk + 0.5)) /
                ((nk - rk + 0.5) / (N - nk - r + rk + 0.5))
            )
            term_weights[term] = weight
        else:
            term_weights[term] = 0

    scores = {}
    for _, row in documents_df.iterrows():
        doc_id = row['docID']
        doc_terms = set(row['processed_text'].split())
        score = sum(term_weights.get(term, 0) for term in processed_query if term in doc_terms)
        scores[doc_id] = score

    results_df = pd.DataFrame(list(scores.items()), columns=['docID', 'score'])
    results_df = results_df.sort_values(by='score', ascending=False).reset_index(drop=True)

    return pd.merge(results_df, documents_df, on='docID')


# --- Example Usage for BIM ---
# This part assumes documents_df has been created and populated from the above cells

# Define a sample query
user_query = "information models"

# --- PHASE 1 ---
# Initial ranking without any relevance information
print(f"--- BIM Phase 1: Initial Ranking for query: '{user_query}' ---")
initial_results = rank_bim_phase1(user_query, documents_df)
display(initial_results[['docID', 'text', 'score']])

# --- USER FEEDBACK ---
# Simulate user feedback: documents 0 and 2 are judged as relevant
relevant_documents_from_user = [0, 2]

# --- PHASE 2 ---
# Refined ranking with user feedback
print(f"\n--- BIM Phase 2: Refined Ranking with feedback on relevant docs: {relevant_documents_from_user} ---")
refined_results = rank_bim_phase2(user_query, documents_df, relevant_documents_from_user)
display(refined_results[['docID', 'text', 'score']])

--- BIM Phase 1: Initial Ranking for query: 'information models' ---


,docID,text,score
0,0,This is the first document about information r...,-0.510826
1,1,The second document discusses vector space mod...,-0.510826
2,2,Information retrieval models include probabili...,-1.021651



--- BIM Phase 2: Refined Ranking with feedback on relevant docs: [0, 2] ---


,docID,text,score
0,0,This is the first document about information r...,2.708050
1,2,Information retrieval models include probabili...,1.609438
2,1,The second document discusses vector space mod...,-1.098612


In [ ]:
from collections import defaultdict

def calculate_tf(documents_df):
    """
    Calculates the Term Frequency for each term in each document.

    Args:
        documents_df (pd.DataFrame): DataFrame with 'docID' and 'processed_text'.

    Returns:
        dict: A dictionary of dictionaries, with TF counts.
    """
    tf_scores = {}
    for _, row in documents_df.iterrows():
        doc_id = row['docID']
        terms = row['processed_text'].split()
        term_counts = defaultdict(int)
        for term in terms:
            term_counts[term] += 1
        tf_scores[doc_id] = dict(term_counts)
    return tf_scores

# Calculate TF for the documents
term_frequencies = calculate_tf(documents_df)
# print("Term Frequencies:")
# print(term_frequencies)

In [ ]:
import math

def calculate_idf(documents_df):
    """
    Calculates the Inverse Document Frequency for each term in the collection.

    Args:
        documents_df (pd.DataFrame): DataFrame with 'docID' and 'processed_text'.

    Returns:
        dict: A dictionary of IDF scores.
    """
    N = len(documents_df)
    idf_scores = {}
    all_terms = set()
    for text in documents_df['processed_text']:
        all_terms.update(text.split())

    for term in all_terms:
        # Count the number of documents containing the term
        doc_count = sum(1 for text in documents_df['processed_text'] if term in text.split())
        # Apply the IDF formula
        idf_scores[term] = math.log(N / doc_count)
    return idf_scores

# Calculate IDF for the terms
inverse_document_frequencies = calculate_idf(documents_df)
# print("\nInverse Document Frequencies:")
# print(inverse_document_frequencies)

In [ ]:
import numpy as np

def calculate_tfidf_matrix(documents_df):
    """
    Builds a custom TF-IDF matrix from the documents.
    """
    # Get all unique terms to form the vocabulary
    all_terms = sorted(list(set(term for text in documents_df['processed_text'] for term in text.split())))
    vocab_map = {term: i for i, term in enumerate(all_terms)}

    # Calculate TF and IDF scores
    tf_scores = calculate_tf(documents_df)
    idf_scores = calculate_idf(documents_df)

    # Initialize the TF-IDF matrix
    num_docs = len(documents_df)
    num_terms = len(all_terms)
    tfidf_matrix = np.zeros((num_docs, num_terms))

    # Populate the matrix with TF-IDF scores
    for i, doc_id in enumerate(documents_df['docID']):
        doc_tf_scores = tf_scores[doc_id]
        for term, tf_value in doc_tf_scores.items():
            j = vocab_map[term]
            tfidf_matrix[i, j] = tf_value * idf_scores.get(term, 0)

    # Return as a DataFrame for easy visualization
    return pd.DataFrame(tfidf_matrix, index=documents_df['docID'], columns=all_terms)

# Build the custom TF-IDF matrix
custom_tfidf_matrix = calculate_tfidf_matrix(documents_df)

print("\nCustom TF-IDF Matrix:")
print(custom_tfidf_matrix)


Custom TF-IDF Matrix:
          about  discusses  document     first   include  information  \
docID                                                                   
0      1.098612   0.000000  0.405465  1.098612  0.000000     0.405465   
1      0.000000   1.098612  0.405465  0.000000  0.000000     0.000000   
2      0.000000   0.000000  0.000000  0.000000  1.098612     0.405465   

         models  probabilistic  retrieval    second     space      this  \
docID                                                                     
0      0.000000       0.000000   0.405465  0.000000  0.000000  1.098612   
1      0.405465       0.000000   0.000000  1.098612  1.098612  0.000000   
2      0.810930       1.098612   0.405465  0.000000  0.000000  0.000000   

         vector  
docID            
0      0.000000  
1      1.098612  
2      0.000000  
